In [3]:
import numpy as np
import pandas as pd
import math
import copy

In [4]:
rating_df_columns = ["UserId", "MovieId", "Rating", "TimeStamp"]
rating_df = pd.read_table("ml-1m/ratings.dat", sep="::", names=rating_df_columns)

/home/vedant/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [5]:
rating_df.drop(index =range(100000,1000209), inplace=True)

In [6]:
rating_matrix = np.asarray(rating_df.pivot(index = "UserId", columns = "MovieId", values = "Rating").fillna(0))

In [7]:
### Mean center

def mean_center(A):
    mean_centered_A = np.copy(A)
    for row in range(A.shape[0]):
        ratings_sum = A[row].sum()
        movies_rated = (A[row] > 0).sum()
        mean = ratings_sum/movies_rated
        for i in A[row].nonzero():
            mean_centered_A[row][i] -= mean
    return mean_centered_A

In [8]:
rating_matrix_centered = mean_center(rating_matrix)

In [9]:
### SVD
# A = np.asarray([[3,1,1],[-1,3,1]])

In [38]:
A = rating_matrix

In [39]:
def svd(A):
    
    transpose_flag = 0
    if A.shape[0] > A.shape[1]:
        transpose_flag = 1
        A = A.T
    
    AAt = np.matmul(A, A.T)
    AtA = np.matmul(A.T, A)
    
    eigenvalue_AAt, eigenvector_AAt = np.linalg.eig(AAt)
    eigenvalue_AtA, eigenvector_AtA = np.linalg.eig(AtA)
    
    eigenvector_AAt_transpose = eigenvector_AAt.T
    eigenvector_AtA_transpose = eigenvector_AtA.T
    
    eigenvector_AAt_transpose[eigenvalue_AAt < 10e-2] = 0
    eigenvector_AtA_transpose[eigenvalue_AtA < 10e-2] = 0
    eigenvalue_AAt[eigenvalue_AAt < 10e-2] = 0
    eigenvalue_AtA[eigenvalue_AtA < 10e-2] = 0
    
    eigenvalue_AAt = np.sqrt(eigenvalue_AAt.real)
    eigenvalue_AtA = np.sqrt(eigenvalue_AtA.real)
    
    eigenvector_AAt = eigenvector_AAt.real
    eigenvector_AtA = eigenvector_AtA.real
    
    argsort_eigenval_AAt = np.argsort(-eigenvalue_AAt)
    argsort_eigenval_AtA = np.argsort(-eigenvalue_AtA)
    
    U = np.zeros(eigenvector_AAt.shape)
    sigma1 = np.sort(eigenvalue_AAt)[::-1]
    
    V = np.zeros(eigenvector_AtA.shape)
    sigma2 = (np.sort(eigenvalue_AtA))[::-1]
    for i,j in enumerate(argsort_eigenval_AtA):
        V[:,i] = eigenvector_AtA[:,j]
        
    sigma = np.zeros(A.shape)
    sigma[:, :A.shape[0]] = np.diag(sigma1)
        
    for i in range(U.shape[1]):
        if sigma1[i] != 0:
            U[:,i] = (A.dot(V[:,i]))/sigma1[i]
        else:
            U[:,i] = 0
        
    if transpose_flag == 0:
        return U, sigma, V.T
    else:
        return V, sigma.T, U.T

In [40]:
U, sigma, Vt = svd(A)
A_pred_new  = U.dot(sigma.dot(Vt))

In [41]:
n_component = 10
sigma_reduced = sigma[:,:]
V_reduced = V[:,:]
A_pred = U.dot(sigma_reduced.dot(V_reduced.T))


In [42]:
# u,s,vh = np.linalg.svd(A)
# S = np.zeros(A.shape)
# S[:A.shape[0], :A.shape[0]] = np.diag(s)
# Apred_svd = u.dot(S.dot(Vh))
# S = S[:,:]
# Vh = vh[:,:]

In [43]:
mse_new = (((A_pred_new - A) ** 2).sum()/(A.shape[0]* A.shape[1]))

In [44]:
print(mse_new)

1.541649329050951e-25